In [1]:
from IPython.display import display, Image, clear_output

# Comprehensive Lipidome Automation Workflow (CLAW)

Welcome to CLAW, a tool designed to facilitate and optimize the processing of lipidomic MRM data. This Jupyter notebook encapsulates a suite of tools that streamline the various stages of lipidomics data analysis.

Our toolset enables users to efficiently process MRM data files in the mzML format. Upload a file and CLAW will parse the data into a structured Pandas dataframe. This dataframe includes critical information like sample_ID, MRM transition, and signal intensity. Furthermore, our tool aligns each MRM transition with a default or custom lipid_database for accurate and swift annotation.

Moreover, CLAW is equipped with an OzESI option, a tool to elucidate the double bond location in lipid isomers. This feature allows users to input OzESI data and pinpoint the precise location of double bonds in isomeric lipids. Users have the flexibility to select which double bond locations they want to analyze. Following this, CLAW autonomously predicts potential m/z values and cross-references these predictions with sample data, ensuring a comprehensive and meticulous analysis.

With automation at its core, CLAW eliminates the need for manual data processing, significantly reducing time expenditure. It is a robust and invaluable tool for handling large volumes of lipid MRM data, accelerating scientific discovery in the field of lipidomics.

In [2]:
# ============================================================================
# IMPORTS
# ============================================================================

# Standard library imports
import os
import csv
import json
import importlib
from pathlib import Path

# Third-party imports
import pymzml
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# CLAW library imports
# import CLAW_OzESI
# import plot_TG
# import plot

# # Reload modules for development
# importlib.reload(CLAW_OzESI)
# importlib.reload(plot_TG)
# importlib.reload(plot)

# Import specific functions/classes
from scripts.CLAW_OzESI import Parse, ion_label_parser
from scripts.plot import plot_n9_n7_ratios, plot_n9_n7_ratios_intensity, calculate_ratio_statistics
from pathlib import Path

## Pre-Parsing Setup

This section parses mzML files and extracts MRM transition data into a structured format for analysis. The `CLAW_OzESI.parsing_mzml_to_df` function reads raw mass spectrometry data, matches it against a lipid database, and organizes the results into a pandas DataFrame.

The function requires several inputs to configure the parsing process:

**Database and file locations:**
- `data_base_name_location`: Path to the lipid database containing lipid MRM transitions
- `Project_Folder_data`: Directory containing the mzML files to be analyzed

**Matching parameters:**
- `tolerance`: Acceptable deviation range (in m/z units) when matching detected transitions to the database
- `remove_std`: When set to `True`, filters out transitions corresponding to internal or external standards

The function returns a pandas DataFrame where each row represents a detected MRM transition. Key columns include the sample ID, MRM transition (parent → product ion), and signal intensity, along with additional metadata for downstream analysis.

In [9]:
# PATHS:
data_base_name_location = "/scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/lipid_database/Lipid_Database.xlsx"
Project_Folder_data = "/scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/Projects/canola_tutorial/mzml/ON"
analysis_dir = "/scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/Projects/canola_tutorial/results"
Project_results = "/scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/Projects/canola_tutorial/results"
file_name_to_save = "parsed_results"
tolerance = 0.3

# Run CLAW parsing

In [10]:


# Create parser with corrected path
parser = Parse(
    data_base_name_location=data_base_name_location,
    Project_Folder_data=Project_Folder_data,
    Project_results=Project_results,
    file_name_to_save=file_name_to_save,
    tolerance=tolerance,
    remove_std=True,
    save_data=False,
    batch_processing=True,
    plot_chromatogram=True
)

# Run the parser
df_MRM, df_OzESI, df_OzESI_matched = parser.mrm_run_all(deuterated=False)

# Check the results
print(f"df_MRM shape: {df_MRM.shape}")
print(f"df_OzESI shape: {df_OzESI.shape}")
print(f"df_OzESI_matched shape: {df_OzESI_matched.shape}")

/scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/scripts/CLAW_OzESI.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lipid_MRM_data['Parent_Ion'] = np.round(lipid_MRM_data['Parent_Ion'], 1)
/scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/scripts/CLAW_OzESI.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lipid_MRM_data['Product_Ion'] = np.round(lipid_MRM_data['Product_Ion'], 1)
/scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA

Finished parsing mzML file: /scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/Projects/canola_tutorial/mzml/ON/CrudeCanola_O3on_150gN3_02082023.mzML

Finished parsing mzML file: /scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/Projects/canola_tutorial/mzml/ON/DegummedCanola_O3on_150gN3_02082023.mzML

Finished parsing mzML file: /scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/Projects/canola_tutorial/mzml/ON/RBDCanola_O3on_150gN3_02082023.mzML

Finished parsing all mzML files



/scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/scripts/CLAW_OzESI.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lipid_MRM_data['Parent_Ion'] = np.round(lipid_MRM_data['Parent_Ion'], 1)
/scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/scripts/CLAW_OzESI.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lipid_MRM_data['Product_Ion'] = np.round(lipid_MRM_data['Product_Ion'], 1)
/scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA

df_MRM shape: (105, 7)
df_OzESI shape: (225357, 7)
df_OzESI_matched shape: (225357, 8)


Define the master dataframes where the data will be stored during the parsing step.

## CLAW.full_parse()
In this code, the `CLAW.full_parse()` function is used to analyze the MRM data. It takes several parameters like the location of the lipid database, paths to the data and results folders, the name of the result files, and the tolerance for MRM transitions matching. The function returns two dataframes: `df_matched` that contains information about each detected lipid species and their corresponding MRM transitions, and `OzESI_time_df` which captures data related to OzESI-MS scans, including potential double bond locations of lipids. If `remove_std` is `True`, it removes MRM transitions related to standards from the dataframe, and if `save_data` is `True`, the dataframe is saved as a .csv file in the specified results folder.

In [11]:
df_MRM.head(None)

,Class,Intensity,Lipid,Parent_Ion,Product_Ion,Sample_ID,Transition
0,NaN,5.451378e+05,NaN,760.6,571.596,CrudeCanola_O3on_150gN3_02082023,760.6 -> 571.596
1,NaN,6.208219e+05,NaN,762.6,573.596,CrudeCanola_O3on_150gN3_02082023,762.6 -> 573.596
2,NaN,9.441859e+05,NaN,764.6,575.596,CrudeCanola_O3on_150gN3_02082023,764.6 -> 575.596
3,NaN,1.137434e+06,NaN,766.7,577.596,CrudeCanola_O3on_150gN3_02082023,766.7 -> 577.596
4,NaN,5.900676e+05,NaN,782.6,593.596,CrudeCanola_O3on_150gN3_02082023,782.6 -> 593.596
...,...,...,...,...,...,...,...
100,TAG,4.897507e+05,[TG(54:6)]_FA18:1,896.8,597.596,RBDCanola_O3on_150gN3_02082023,896.8 -> 597.596
101,TAG,1.179904e+06,[TG(54:5)]_FA18:1,898.8,599.596,RBDCanola_O3on_150gN3_02082023,898.8 -> 599.596
102,TAG,1.654774e+06,"[TG(55:11),TG(54:4)]_FA18:1",900.8,601.596,RBDCanola_O3on_150gN3_02082023,900.8 -> 601.596
103,TAG,5.234119e+06,"[TG(55:10),TG(54:3)]_FA18:1",902.8,603.596,RBDCanola_O3on_150gN3_02082023,902.8 -> 603.596


The `read_mrm_list()` function is first invoked to read the MRM database from the specified file location and return it as a pandas DataFrame `mrm_database`. Subsequently, the `match_lipids_parser()` function is called to match the detected lipids from the `OzESI_time_df` DataFrame, obtained from the OzESI-MS scans, with the known lipids in the `mrm_database` based on the MRM transitions within the specified `tolerance`. The result is saved in the `df_oz_matched` DataFrame, which now contains matched lipid species from the OzESI-MS data.

### Confirm OzESI df before parsing data

In [12]:
df_OzESI

,Lipid,Parent_Ion,Product_Ion,Retention_Time,OzESI_Intensity,Sample_ID,Transition
0,NaN,760.6,571.596,0.015933,190.400009,CrudeCanola_O3on_150gN3_02082023,760.6 -> 571.596
1,NaN,760.6,571.596,0.032233,147.420013,CrudeCanola_O3on_150gN3_02082023,760.6 -> 571.596
2,NaN,760.6,571.596,0.048550,153.620010,CrudeCanola_O3on_150gN3_02082023,760.6 -> 571.596
3,NaN,760.6,571.596,0.064850,200.080017,CrudeCanola_O3on_150gN3_02082023,760.6 -> 571.596
4,NaN,760.6,571.596,0.081167,206.900009,CrudeCanola_O3on_150gN3_02082023,760.6 -> 571.596
...,...,...,...,...,...,...,...
225352,NaN,904.8,605.596,34.931700,148.500015,RBDCanola_O3on_150gN3_02082023,904.8 -> 605.596
225353,NaN,904.8,605.596,34.948000,131.800003,RBDCanola_O3on_150gN3_02082023,904.8 -> 605.596
225354,NaN,904.8,605.596,34.964317,151.960007,RBDCanola_O3on_150gN3_02082023,904.8 -> 605.596
225355,NaN,904.8,605.596,34.980617,137.700012,RBDCanola_O3on_150gN3_02082023,904.8 -> 605.596


In [13]:
df_OzESI_matched.to_csv('df_OzESI_matched.csv')
df_OzESI_matched


,Class,Lipid,OzESI_Intensity,Parent_Ion,Product_Ion,Retention_Time,Sample_ID,Transition
0,NaN,NaN,190.400009,760.6,571.596,0.015933,CrudeCanola_O3on_150gN3_02082023,760.6 -> 571.596
1,NaN,NaN,147.420013,760.6,571.596,0.032233,CrudeCanola_O3on_150gN3_02082023,760.6 -> 571.596
2,NaN,NaN,153.620010,760.6,571.596,0.048550,CrudeCanola_O3on_150gN3_02082023,760.6 -> 571.596
3,NaN,NaN,200.080017,760.6,571.596,0.064850,CrudeCanola_O3on_150gN3_02082023,760.6 -> 571.596
4,NaN,NaN,206.900009,760.6,571.596,0.081167,CrudeCanola_O3on_150gN3_02082023,760.6 -> 571.596
...,...,...,...,...,...,...,...,...
225352,TAG,"[TG(55:9),TG(54:2)]_FA18:1",148.500015,904.8,605.596,34.931700,RBDCanola_O3on_150gN3_02082023,904.8 -> 605.596
225353,TAG,"[TG(55:9),TG(54:2)]_FA18:1",131.800003,904.8,605.596,34.948000,RBDCanola_O3on_150gN3_02082023,904.8 -> 605.596
225354,TAG,"[TG(55:9),TG(54:2)]_FA18:1",151.960007,904.8,605.596,34.964317,RBDCanola_O3on_150gN3_02082023,904.8 -> 605.596
225355,TAG,"[TG(55:9),TG(54:2)]_FA18:1",137.700012,904.8,605.596,34.980617,RBDCanola_O3on_150gN3_02082023,904.8 -> 605.596


# Match user selected ion labels with OzESI_Matched Data using m/z and retention time tolerances (±0.3 Da, ±1.0 min)

In [12]:
# Call the function

lipid_mrm = pd.read_csv("Projects/canola_tutorial/results/lipid_mrm.csv")
df_canola_lipids = ion_label_parser(
    lipid_mrm=lipid_mrm,
    df_OzESI_matched=df_OzESI_matched,
    ion_tolerance=0.3,
    rt_tolerance=1.0,
    ion_labels=['n-7', 'n-9']  # Select double bond locations
)

In [13]:
# DIAGNOSTIC: Check why ion_label_parser returns empty
print("=" * 60)
print("DEBUGGING ion_label_parser")
print("=" * 60)

# Check lipid_mrm
print("\n1. lipid_mrm DataFrame:")
print(f"   Shape: {lipid_mrm.shape}")
print(f"   Columns: {lipid_mrm.columns.tolist()}")
print("\n   First few rows:")
print(lipid_mrm.head())

# Check df_OzESI_matched
print("\n2. df_OzESI_matched DataFrame:")
print(f"   Shape: {df_OzESI_matched.shape}")
print(f"   Columns: {df_OzESI_matched.columns.tolist()}")
if len(df_OzESI_matched) > 0:
    print(f"   Sample_IDs: {df_OzESI_matched['Sample_ID'].unique()}")
    print(f"   Parent_Ion range: {df_OzESI_matched['Parent_Ion'].min():.1f} - {df_OzESI_matched['Parent_Ion'].max():.1f}")
    print(f"   Product_Ion range: {df_OzESI_matched['Product_Ion'].min():.1f} - {df_OzESI_matched['Product_Ion'].max():.1f}")
    print(f"   RT range: {df_OzESI_matched['Retention_Time'].min():.2f} - {df_OzESI_matched['Retention_Time'].max():.2f}")
    print("\n   First few rows:")
    print(df_OzESI_matched.head())
else:
    print("   ⚠️ df_OzESI_matched is EMPTY!")

# Manual test of one match
if len(lipid_mrm) > 0 and len(df_OzESI_matched) > 0:
    print("\n3. Testing first lipid match:")
    test_row = lipid_mrm.iloc[0]
    print(f"   Looking for: {test_row['Lipid']}")
    print(f"   Product_Ion: {test_row['Product_Ion']}")
    print(f"   RT: {test_row['Retention_Time']}")
    print(f"   n-7 ion: {test_row['n-7']}")
    print(f"   n-9 ion: {test_row['n-9']}")
    
    # Check if any matches exist
    matches = df_OzESI_matched[
        (df_OzESI_matched['Product_Ion'].between(test_row['Product_Ion'] - 0.3, test_row['Product_Ion'] + 0.3))
    ]
    print(f"\n   Product ion matches: {len(matches)}")
    if len(matches) > 0:
        print(matches[['Parent_Ion', 'Product_Ion', 'Retention_Time', 'Sample_ID']].head())

print("=" * 60)

DEBUGGING ion_label_parser

1. lipid_mrm DataFrame:
   Shape: (7, 6)
   Columns: ['Lipid', 'Precursor_Ion', 'Product_Ion', 'n-7', 'n-9', 'Retention_Time']

   First few rows:
               Lipid  Precursor_Ion  Product_Ion    n-7    n-9  Retention_Time
0  [TG(52:2)]_FA18:1          876.8      577.596  794.8  766.8           18.05
1  [TG(52:3)]_FA18:1          874.8      575.596  792.8  764.8           16.09
2  [TG(52:4)]_FA18:1          872.8      573.596  790.8  762.8           14.30
3  [TG(54:2)]_FA18:1          904.8      605.596  822.8  794.8           20.00
4  [TG(54:3)]_FA18:1          902.8      603.596  820.8  792.8           17.98

2. df_OzESI_matched DataFrame:
   Shape: (225357, 8)
   Columns: ['Class', 'Lipid', 'OzESI_Intensity', 'Parent_Ion', 'Product_Ion', 'Retention_Time', 'Sample_ID', 'Transition']
   Sample_IDs: ['CrudeCanola_O3on_150gN3_02082023' 'DegummedCanola_O3on_150gN3_02082023'
 'RBDCanola_O3on_150gN3_02082023']
   Parent_Ion range: 760.6 - 904.8
   Product_Ion

In [14]:
df_canola_lipids

,Lipid,db_pos,Parent_Ion,Product_Ion,Retention_Time,OzESI_Intensity,Sample_ID,Transition
0,[TG(52:2)]_FA18:1,n-7,794.7,577.596,17.050733,552.000061,CrudeCanola_O3on_150gN3_02082023,794.7 -> 577.596
1,[TG(52:2)]_FA18:1,n-7,794.7,577.596,17.067050,586.240051,CrudeCanola_O3on_150gN3_02082023,794.7 -> 577.596
2,[TG(52:2)]_FA18:1,n-7,794.7,577.596,17.083350,489.380035,CrudeCanola_O3on_150gN3_02082023,794.7 -> 577.596
3,[TG(52:2)]_FA18:1,n-7,794.7,577.596,17.099667,442.220032,CrudeCanola_O3on_150gN3_02082023,794.7 -> 577.596
4,[TG(52:2)]_FA18:1,n-7,794.7,577.596,17.115967,345.980011,CrudeCanola_O3on_150gN3_02082023,794.7 -> 577.596
...,...,...,...,...,...,...,...,...
5161,[TG(54:5)]_FA18:1,n-9,788.6,599.596,15.243367,802.380066,RBDCanola_O3on_150gN3_02082023,788.6 -> 599.596
5162,[TG(54:5)]_FA18:1,n-9,788.6,599.596,15.259667,786.740051,RBDCanola_O3on_150gN3_02082023,788.6 -> 599.596
5163,[TG(54:5)]_FA18:1,n-9,788.6,599.596,15.275983,563.920044,RBDCanola_O3on_150gN3_02082023,788.6 -> 599.596
5164,[TG(54:5)]_FA18:1,n-9,788.6,599.596,15.292283,703.860046,RBDCanola_O3on_150gN3_02082023,788.6 -> 599.596


# Analyze lipid peaks for selected sample and save n-9/n-7 ratio summary table

In [ ]:
SHOW_PLOTS = False  # Set True to display plots

# =====================================================================
# RUN PEAK ANALYSIS
# =====================================================================

summary_df = CLAW_OzESI.peak_analysis(
    df_canola_lipids,
    output_dir=str(analysis_dir),
    show_plots=SHOW_PLOTS,
    plot_TG_module=plot_TG
)



Found 3 samples:
 • CrudeCanola_O3on_150gN3_02082023
 • DegummedCanola_O3on_150gN3_02082023
 • RBDCanola_O3on_150gN3_02082023

🔍 Processing sample: CrudeCanola_O3on_150gN3_02082023

Peak Analysis for \[TG\(52:2\)\]_FA18:1 in CrudeCanola_O3on_150gN3_02082023:
--------------------------------------------------
n-7 Largest Peak Area: 385.16
  Retention Time: 18.05
  Peak Height: 3814.96
  All 1 peaks:
    Peak 1 - RT: 18.05, Height: 3814.96, Area: 385.16
n-9 Largest Peak Area: 1594.45
  Retention Time: 18.05
  Peak Height: 16153.12
  All 1 peaks:
    Peak 1 - RT: 18.05, Height: 16153.12, Area: 1594.45

n-9/n-7 Ratio (largest peaks only): 4.1397

Peak Analysis for \[TG\(52:3\)\]_FA18:1 in CrudeCanola_O3on_150gN3_02082023:
--------------------------------------------------
n-7 Largest Peak Area: 181.90
  Retention Time: 16.04
  Peak Height: 1083.40
  All 1 peaks:
    Peak 1 - RT: 16.04, Height: 1083.40, Area: 181.90
n-9 Largest Peak Area: 403.31
  Retention Time: 16.09
  Peak Height: 3419.8

,Sample_ID,Lipid,n-7_Area,n-9_Area,n-9/n-7_Ratio,n-7_Intensity,n-9_Intensity,n-9/n-7_Intensity_Ratio
0,CrudeCanola_O3on_150gN3_02082023,[TG(52:2)]_FA18:1,385.164882,1594.448408,4.139652,3814.960205,16153.121094,4.234152
1,CrudeCanola_O3on_150gN3_02082023,[TG(52:3)]_FA18:1,181.897408,403.309804,2.217238,1083.400024,3419.820312,3.156563
2,CrudeCanola_O3on_150gN3_02082023,[TG(52:4)]_FA18:1,113.472247,255.856166,2.254791,615.420044,1874.780151,3.046342
3,CrudeCanola_O3on_150gN3_02082023,[TG(54:2)]_FA18:1,217.319776,1178.185661,5.421438,2716.400146,11693.081055,4.304624
4,CrudeCanola_O3on_150gN3_02082023,[TG(54:3)]_FA18:1,1585.371000,9933.026601,6.265427,17186.220703,99860.148438,5.810477


# Plot CLAW vs Manual Comparisons

In [16]:
from pathlib import Path

# Convert Project_results to Path object
Project_results = Path("/scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/Projects/canola_tutorial/results")


SAMPLES = [
    {'name': 'Crude', 'sample_file': 'df_CrudeCanola_O3on_150gN3_02082023_summary.csv', 
     'manual_file': 'df_manual_crude.csv'},
    {'name': 'Degummed', 'sample_file': 'df_DegummedCanola_O3on_150gN3_02082023_summary.csv', 
     'manual_file': 'df_manual_degummed.csv'},
    {'name': 'RBD', 'sample_file': 'df_RBDCanola_O3on_150gN3_02082023_summary.csv', 
     'manual_file': 'df_manual_RBD.csv'}
]

# Create plot directories
(Project_results / "ratio_area").mkdir(parents=True, exist_ok=True)
(Project_results / "ratio_intensity").mkdir(parents=True, exist_ok=True)

for sample in SAMPLES:
    sample_csv = Project_results / sample['sample_file']
    manual_csv = Project_results / sample['manual_file']
    
    if sample_csv.exists() and manual_csv.exists():
        # Area-based ratios
        plot_n9_n7_ratios(
            sample_csv_path=str(sample_csv),
            manual_csv_path=str(manual_csv),
            sample_label=f'{sample["name"]} CLAW Area',
            manual_label=f'{sample["name"]} Manual Area',
            file_path=str(Project_results / "ratio_area" / f"{sample['name']}_area_ratios")
        )
        
        # Intensity-based ratios
        plot_n9_n7_ratios_intensity(
            sample_csv_path=str(sample_csv),
            manual_csv_path=str(manual_csv),
            sample_label=f'{sample["name"]} CLAW Intensity',
            manual_label=f'{sample["name"]} Manual Area',
            file_path=str(Project_results / "ratio_intensity" / f"{sample['name']}_intensity_ratios")
        )

print(f"✓ Area plots saved to: {Project_results / 'ratio_area'}")
print(f"✓ Intensity plots saved to: {Project_results / 'ratio_intensity'}")

Saved PNG: /scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/Projects/canola_tutorial/results/ratio_area/Crude_area_ratios.png
Saved PDF: /scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/Projects/canola_tutorial/results/ratio_area/Crude_area_ratios.pdf
Saved sample CSV: /scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/Projects/canola_tutorial/results/ratio_area/Crude_area_ratios_sample_df_20251105.csv
Saved manual CSV: /scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/Projects/canola_tutorial/results/ratio_area/Crude_area_ratios_manual_df_20251105.csv
Saved PNG: /scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/Projects/canola_tutorial/results/ratio_intensity/Crude_intensity_ratios.png
Saved PDF: /scratch/negishi/iyer95/iyer95/CLAW_OzESI_Paper/OzESI_MUFA_paper/lipid_platform/Projects/canola_tutorial/results/ratio_intensity/Crude_intensi